### Importing What We Need (Libs)

In [73]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

### Step 1 : Cleaning The Data For Furthur Processing & Feature Selection

In [74]:
## Reading The Data From our dataset
raw_data_frame = pd.read_csv('./Data Set/train.csv')
raw_data_frame.head()

,title,year,mileage,transmission,fuel,body_color,inside_color,body_status,description,body_type,volume,engine,acceleration,price
0,هیوندای، توسان,2014,"175,000 km",اتوماتیک,9.6 لیتر در صد کیلومتر,سفید,مارون,بدون رنگ,مانیتور اندرویدی بدنه بدون لیسه کشی و روغن گیر...,crossover,2.4 لیتر,4 سیلندر,10.4 ثانیه,"2,550,000,000"
1,پراید، 131,1391,"330,000 km",دنده ای,6.7 لیتر در صد کیلومتر,سفید,سفید,بدون رنگ,ماشین شاستی سرشاستی پلمپ میباشد به شرط موتوری ...,passenger_car,1.3 لیتر,4 سیلندر یورو3 (M13),13.8 ثانیه,"215,000,000"
2,پژو، 206,1398,"132,000 km",دنده ای,6.4 لیتر در صد کیلومتر,سفید,خاکستری,بدون رنگ,خیلی تمیز فنی عالی,hatchback,1.4 لیتر,4 سیلندر TU3,14.1 ثانیه,"445,000,000"
3,تیبا، هاچ بک,1396,"103,000 km",دنده ای,6.9 لیتر در صد کیلومتر,آبی,خاکستری,بدون رنگ,ماشین دارای دوربین دنده عقب،روکش صندلی نو،زاپا...,hatchback,1.5 لیتر,4 سیلندر M15,13 ثانیه,"310,000,000"
4,پژو، پارس,1401,"37,000 km",دنده ای,7 لیتر در صد کیلومتر,خاکستری,کرم,بدون رنگ,NaN,passenger_car,1.8 لیتر,4 سیلندر XU7 پلاس,12 ثانیه,"570,000,000"


In [75]:
## Finding Null Values In Each Column
raw_data_frame.isnull().sum()

title              0
year               0
mileage            0
transmission       0
fuel            1657
body_color         0
inside_color       0
body_status        0
description     2098
body_type          0
volume          1217
engine          1217
acceleration    1836
price              0
dtype: int64

In [76]:
# Drop rows where the value in 'fuel' is null
df_dropped_na_values_in_rows = raw_data_frame.dropna(subset=['fuel'], inplace=True)
df_dropped_na_values_in_rows = raw_data_frame.dropna(subset=['description'], inplace=True)
df_dropped_na_values_in_rows = raw_data_frame.dropna(subset=['volume'], inplace=True)
df_dropped_na_values_in_rows = raw_data_frame.dropna(subset=['engine'], inplace=True)
df_dropped_na_values_in_rows = raw_data_frame.dropna(subset=['acceleration'], inplace=True)



df = raw_data_frame
print(f"Null Value Counts In Columns(fuel,volume,engine): {df['fuel'].isnull().sum()}, {df['volume'].isnull().sum()}, {df['engine'].isnull().sum()}")


Null Value Counts In Columns(fuel,volume,engine): 0, 0, 0


In [77]:
df.head()

,title,year,mileage,transmission,fuel,body_color,inside_color,body_status,description,body_type,volume,engine,acceleration,price
0,هیوندای، توسان,2014,"175,000 km",اتوماتیک,9.6 لیتر در صد کیلومتر,سفید,مارون,بدون رنگ,مانیتور اندرویدی بدنه بدون لیسه کشی و روغن گیر...,crossover,2.4 لیتر,4 سیلندر,10.4 ثانیه,"2,550,000,000"
1,پراید، 131,1391,"330,000 km",دنده ای,6.7 لیتر در صد کیلومتر,سفید,سفید,بدون رنگ,ماشین شاستی سرشاستی پلمپ میباشد به شرط موتوری ...,passenger_car,1.3 لیتر,4 سیلندر یورو3 (M13),13.8 ثانیه,"215,000,000"
2,پژو، 206,1398,"132,000 km",دنده ای,6.4 لیتر در صد کیلومتر,سفید,خاکستری,بدون رنگ,خیلی تمیز فنی عالی,hatchback,1.4 لیتر,4 سیلندر TU3,14.1 ثانیه,"445,000,000"
3,تیبا، هاچ بک,1396,"103,000 km",دنده ای,6.9 لیتر در صد کیلومتر,آبی,خاکستری,بدون رنگ,ماشین دارای دوربین دنده عقب،روکش صندلی نو،زاپا...,hatchback,1.5 لیتر,4 سیلندر M15,13 ثانیه,"310,000,000"
6,پژو، 207,1400,"95,000 km",دنده ای,6.6 لیتر در صد کیلومتر,مشکی,نوک مدادی,دو لکه رنگ,فرمان برقی گلگیر و درب عقب سمت راننده رنگ دارد...,hatchback,1.6 لیتر,4 سیلندر TU5,11.6 ثانیه,"590,000,000"


In [78]:
# Function to extract the numeric value and convert to float
def extract_float(value):
    try:
        return float(value.split()[0])
    except ValueError:
        return None

## Convert String Into Int
def convert_to_int(string):
    try:
        # Remove the " km" part and replace commas with empty strings
        number_string = string.replace(" km", "").replace(",", "")

        # Convert the resulting string to an integer
        return int(number_string)

    except ValueError:
        # If the conversion fails (e.g., due to invalid characters), return None
        return None
## extract the number in engine column   
def extract_target_number(string):
    import re

    # Use regular expression to find the first digit
    match = re.search(r"\d+", string)

    if match:
        # Extract the matched digit and convert it to an integer
        return int(match.group())

    else:
        # If no digit is found, return None
        return None


# Apply the function to the all in need of dtype convertion column and create a new column
df['fuel'] = df['fuel'].apply(extract_float)
df['volume'] = df['volume'].apply(extract_float)
df['mileage'] = df['mileage'].apply(convert_to_int)
df['price'] = df['price'].apply(convert_to_int)
df['engine'] = df['engine'].apply(extract_target_number)
df['acceleration'] = df['acceleration'].apply(extract_target_number)


## Convert The Columns which contain numeric values into numeric values
le = LabelEncoder()
df['title'] = le.fit_transform(df['title'])
df['transmission'] = le.fit_transform(df['transmission'])
df['body_color'] = le.fit_transform(df['body_color'])
df['body_status'] = le.fit_transform(df['body_status'])
df['body_type'] = le.fit_transform(df['body_type'])
df['inside_color'] = le.fit_transform(df['inside_color'])
df['description'] = le.fit_transform(df['description'])


df.head(2)

,title,year,mileage,transmission,fuel,body_color,inside_color,body_status,description,body_type,volume,engine,acceleration,price
0,141,2014,175000.0,0,9.6,19,26,1,9876,1,2.4,4.0,10,2550000000
1,152,1391,330000.0,1,6.7,19,17,1,9298,3,1.3,4.0,13,215000000


### Step 2 : Create a new data set with the current data set and save the changes

In [79]:
# # Creating a new sata set using the existing data set
# cleaned_df = pd.DataFrame(df)
# cleaned_df.to_csv("finalproccesseddf.csv", index=False)


## Only Run one time to prevent making the new Dataframe again !!!!

In [80]:
# Importing The New Data Farme
readyDf = pd.read_csv('./Data Set/finalproccesseddf_train.csv')
readyDf.head(2)

,title,year,mileage,transmission,fuel,body_color,inside_color,body_status,description,body_type,volume,engine,acceleration,price
0,141,2014,175000.0,0,9.6,19,26,1,9876,1,2.4,4.0,10,2550000000
1,152,1391,330000.0,1,6.7,19,17,1,9298,3,1.3,4.0,13,215000000


### Step 3 : Creating The Price Predictor Machine Learning Model

In [81]:
## Load The test and training data
train_df = pd.read_csv('./Data Set/finalproccesseddf_train.csv')
test_df = pd.read_csv('./Data Set/finaltest.csv')

# Assuming "price" is the target variable (check if it exists in train_df)
y_train = train_df["price"]  # Target variable

# Use all features from train_df (excluding "price" is handled implicitly)
X_train = train_df.drop("price", axis=1)  # Implicitly removes price for features

# Create a Random Forest Regressor model
model = RandomForestRegressor()

# Train the model
model.fit(X_train, y_train)

# Now you can make predictions on the test data, removing "price" if necessary
if "price" in test_df.columns:
    test_df = test_df.drop("price", axis=1)

predictions = model.predict(test_df)

# Fill the price column in the test data
test_df["price"] = predictions

# Save the results to a new CSV file
test_df.to_csv("predictions.csv", index=False)

# Print accuracy
accuracy_train = model.score(X_train, y_train)
print(f"Accuracy on training data: {accuracy_train * 100} %")

Accuracy on training data: 99.50548185035383 %
